In [3]:
from datetime import datetime
print("Current Date/Time: ", datetime.now())

Current Date/Time:  2020-09-26 13:40:43.500542


# Import Files and Packages

In [4]:
from torchsummary import summary

from cuda import initialize_cuda
from data.dataset import CIFAR10
from model import QuizDNN
from model.utils.loss import cross_entropy_loss
from model.utils.optimizers import sgd_optimizer
from model.utils.callbacks import lr_scheduler
from train import train
from evaluate import val

# Initialize Hyperparameters

In [5]:
class Args:

    # Data Loading
    # ============

    train_batch_size = 64
    val_batch_size = 64
    num_workers = 4

    # Training
    # ========
    random_seed = 1
    epochs = 10
    learning_rate = 0.01
    momentum = 0.9
    sample_count = 10

## Set Seed and Get GPU Availability

In [6]:
# Initialize CUDA and set random seed
cuda, device = initialize_cuda(Args.random_seed)

GPU Available? True


# Download CIFAR-10 Dataset

In [7]:
dataset = CIFAR10(
    train_batch_size=Args.train_batch_size,
    val_batch_size=Args.val_batch_size,
    cuda=cuda,
    num_workers=Args.num_workers
)

Extracting /src/D/Research/EVA5-Vision-Squad/S9/data/cifar10/cifar-10-python.tar.gz to /src/D/Research/EVA5-Vision-Squad/S9/data/cifar10
Files already downloaded and verified
Files already downloaded and verified


## Training and Validation Dataloaders

In [8]:
# Create train data loader
train_loader = dataset.loader(train=True)

# Create val data loader
val_loader = dataset.loader(train=False)

# Model Architecture and Summary

In [9]:
model = QuizDNN().to(device)
summary(model, dataset.image_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
              ReLU-2           [-1, 64, 32, 32]               0
       BatchNorm2d-3           [-1, 64, 32, 32]             128
            Conv2d-4           [-1, 64, 32, 32]          36,928
              ReLU-5           [-1, 64, 32, 32]               0
       BatchNorm2d-6           [-1, 64, 32, 32]             128
            Conv2d-7           [-1, 64, 32, 32]          36,928
              ReLU-8           [-1, 64, 32, 32]               0
       BatchNorm2d-9           [-1, 64, 32, 32]             128
        MaxPool2d-10           [-1, 64, 16, 16]               0
           Conv2d-11           [-1, 64, 16, 16]          36,928
             ReLU-12           [-1, 64, 16, 16]               0
      BatchNorm2d-13           [-1, 64, 16, 16]             128
           Conv2d-14           [-1, 64,

# Model Training and Validation

In [10]:
losses = []
accuracies = []
correct_samples = []
incorrect_samples = []

criterion = cross_entropy_loss()  # Create loss function
optimizer = sgd_optimizer(model, Args.learning_rate, Args.momentum)  # Create optimizer
last_epoch = False

for epoch in range(1, Args.epochs + 1):
    print(f'Epoch {epoch}:')
    if epoch == Args.epochs:
        last_epoch = True
    train(model, train_loader, device, optimizer, criterion)
    val(
        model, val_loader, device, criterion, losses, accuracies,
        correct_samples, incorrect_samples, Args.sample_count, last_epoch=last_epoch
    )

  0%|          | 0/782 [00:00<?, ?it/s]

Epoch 1:


  0%|          | 0/782 [00:00<?, ?it/s]


Validation set: Average loss: 0.0171, Accuracy: 6141/10000 (61.41%)

Epoch 2:


  0%|          | 0/782 [00:00<?, ?it/s]

...


Validation set: Average loss: 0.0096, Accuracy: 8090/10000 (80.90%)

Epoch 9:


  0%|          | 0/782 [00:00<?, ?it/s]


Validation set: Average loss: 0.0099, Accuracy: 8069/10000 (80.69%)

Epoch 10:


Loss=0.35 Batch_ID=781 Accuracy=93.46: 100%|██████████| 782/782 [00:09<00:00, 82.52it/s]



Validation set: Average loss: 0.0100, Accuracy: 8147/10000 (81.47%)

